In [41]:
import os
from glob import glob

from typing import List, Tuple

import numpy as np
import pandas as pd
from pandas import DataFrame

# Combining DeepScenario Datasets
Download data from here: https://github.com/Simula-COMPLEX/DeepScenario/tree/main/deepscenario-dataset

In [42]:
STRATEGIES = ["greedy", "random", "rl_based"]
EXT = "*.csv"
DATASET_PATH = f'./DeepScenario/deepscenario-dataset/'

In [43]:
def Load_Files(strategy: str = "greedy") -> List[Tuple[str, str, List[str]]]:
    return [( path, path.split("/")[-1].split("-")[1], files, strategy) 
            for path, subdir, files in os.walk(DATASET_PATH + f'{strategy}-strategy/')
            for file in glob(os.path.join(path, EXT))]

In [52]:
print(Load_Files()[0]) # if this cell fails, you need to download the dataset

('./DeepScenario/deepscenario-dataset/greedy-strategy/reward-dto', 'dto', ['road1-rain_day-scenario-attributes.csv', 'road1-rain_night-scenario-attributes.csv', 'road1-sunny_day-scenario-attributes.csv', 'road1-sunny_night-scenario-attributes.csv', 'road2-rain_day-scenario-attributes.csv', 'road2-rain_night-scenario-attributes.csv', 'road2-sunny_day-scenario-attributes.csv', 'road2-sunny_night-scenario-attributes.csv', 'road3-rain_day-scenario-attributes.csv', 'road3-rain_night-scenario-attributes.csv', 'road3-sunny_day-scenario-attributes.csv', 'road3-sunny_night-scenario-attributes.csv', 'road4-rain_day-scenario-attributes.csv', 'road4-rain_night-scenario-attributes.csv', 'road4-sunny_day-scenario-attributes.csv', 'road4-sunny_night-scenario-attributes.csv'], 'greedy')


In [45]:
def Add_Meta_Data(df: DataFrame, reward: str, road: str, strategy: str, scenario: str):
    df['reward'] = reward
    df['road'] = road
    df['strategy'] = strategy
    df["scenario"] = scenario

def Load_Data(strategy: str = "all") -> DataFrame:
    df = None
    try:
        if strategy == "all":
            datafiles = []
            for s in STRATEGIES:
                datafiles.extend(Load_Files(s)) 
        else:
            if strategy not in STRATEGIES:
                raise ValueError(f"Invalid strategy: {strategy}")
            datafiles = Load_Files(strategy)
        
        for path, reward, files, strat in datafiles:
            for file in files:
                road, scenario, _, _ = file.split("-")
                if df is None:
                    df = pd.read_csv(path + "/" + file)
                    Add_Meta_Data(df, reward, road, strat, scenario)
                else:
                    temp = pd.read_csv(path + "/" + file)
                    Add_Meta_Data(temp, reward, road, strat, scenario)
                    df = pd.concat([df, temp], ignore_index=True)
    except:
        print("Not Found")
    finally:
        return df

In [46]:
df = Load_Data()
print(df.shape)

(536480, 13)


In [47]:
df.head()

,Execution,ScenarioID,Configuration_API_Description,Attribute[TTC],Attribute[DTO],Attribute[Jerk],Attribute[COL],Attribute[COLT],Attribute[SAC],reward,road,strategy,scenario
0,0,0_scenario_0,A red BoxTruck is overtaking (near) the ego ve...,100000.000000,24.810964,3.48,False,None,0.0,dto,road1,greedy,rain_day
1,0,0_scenario_1,A skyblue SUV is driving from the opposite dir...,100000.000000,8.654072,1.04,False,None,0.0,dto,road1,greedy,rain_day
2,0,0_scenario_2,A skyblue BoxTruck is driving ahead (near) of ...,2.103269,10.209357,4.98,False,None,0.0,dto,road1,greedy,rain_day
3,0,0_scenario_3,A black SchoolBus is driving ahead (near) of t...,7.452645,17.011425,2.42,False,None,0.0,dto,road1,greedy,rain_day
4,0,0_scenario_4,A red Hatchback is crossing the road (far) and...,2.885993,27.858975,2.84,False,None,0.0,dto,road1,greedy,rain_day


In [48]:
df["strategy"].unique()

array(['greedy', 'random', 'rl_based'], dtype=object)

In [49]:
df["reward"].value_counts()

jerk    185424
dto     176448
ttc     174608
Name: reward, dtype: int64

In [50]:
df["road"].value_counts()

road3    137904
road2    137440
road1    136896
road4    124240
Name: road, dtype: int64

In [51]:
df["scenario"].value_counts()

sunny_night    135392
rain_night     134384
rain_day       134048
sunny_day      132656
Name: scenario, dtype: int64

In [ ]:
df.to_csv("./data/deep-scenario.csv", index=False)